In [ ]:
!pip install python-dotenv
!pip install azure.identity
!pip install semantic-link-sempy
!pip install azure-storage-blob
!pip install azure-storage-file-datalake

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
APP_REGISTRATION_CLIENT_ID=os.getenv("APP_REGISTRATION_CLIENT_ID")
APP_REGISTRATION_TENANT_ID= os.getenv("APP_REGISTRATION_TENANT_ID")
APP_REGISTRATION_SECRET_VALUE = os.getenv("APP_REGISTRATION_SECRET_VALUE")
FABRIC_WORKSPACE_ID = os.getenv("FABRIC_WORKSPACE_ID")

In [ ]:
warehouse_payload = { 
  "type": "warehouse", 
  "displayName": "warehouse", 
  "description": "New Lakehouse Service Principal case-insensitive collation", 
  "creationPayload": { 
    "defaultCollation": "Latin1_General_100_CI_AS_KS_WS_SC_UTF8" 
  } 
}

In [ ]:
import json, requests, pandas as pd 
import datetime  
from azure.identity import ClientSecretCredential 
 

# Generates the access token for the Service Principal 
api = 'https://analysis.windows.net/powerbi/api/.default' 
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/', 
               tenant_id = APP_REGISTRATION_TENANT_ID, 
               client_id = APP_REGISTRATION_CLIENT_ID, 
               client_secret = APP_REGISTRATION_SECRET_VALUE) 
access_token = auth.get_token(api)
access_token = access_token.token 
## This is where I store my header with the Access Token, because this is required when authenticating 
## to the Power BI Admin APIs 
auth = {'Authorization': f'Bearer {access_token}'} 

In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException

client=fabric.FabricRestClient()

url=f"v1/workspaces/{FABRIC_WORKSPACE_ID}/items"

def create_fabric_item(payload):
    try:
        result=client.post(url,headers=auth,json=payload)
        display(result)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        print(f"Response content: {result.text}")  # Print the response body
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")

In [ ]:
create_fabric_item(warehouse_payload)